# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import os
import json
from pprint import pprint


# Import API key
from api_keys import geoapify_key
gmaps.configure(api_key = geoapify_key)

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hasaki,35.7333,140.8333,3.22,74,100,1.51,JP,1703876157
1,1,Desaguadero,-16.5656,-69.0417,17.22,41,93,6.62,PE,1703876158
2,2,Cockburn Town,21.4612,-71.1419,26.98,74,75,3.09,TC,1703876158
3,3,Coahuayana de Hidalgo,18.7000,-103.6583,28.32,50,0,2.38,MX,1703876158
4,4,Puerto Ayora,-0.7393,-90.3518,29.98,91,53,3.13,EC,1703875896


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

# Display the map
city_data_df = city_data_df.dropna()
humidity = city_data_df["Humidity"].astype(float)
locations = city_data_df[["Lat", "Lng"]].astype(float)

fig_map = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig_map.add_layer(heat_layer)

fig_map

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

#A max temperature lower than 27 degrees but higher than 21
filtered_city_data_df = filtered_city_data_df.loc[(filtered_city_data_df["Max Temp"] < 27) & (filtered_city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
filtered_city_data_df = filtered_city_data_df.loc[filtered_city_data_df["Wind Speed"] < 4.5]
#Zero cloudiness
filtered_city_data_df = filtered_city_data_df.loc[filtered_city_data_df["Cloudiness"] == 0]

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,Timbuktu,16.7735,-3.0074,24.50,11,0,4.16,ML,1703876193
119,119,Dapchi,12.4904,11.4944,22.74,24,0,3.90,NG,1703876198
165,165,Pisco,-13.7000,-76.2167,25.03,69,0,3.60,PE,1703876213
316,316,Duekoué,6.7472,-7.3496,26.40,26,0,0.56,CI,1703876270
324,324,Ouésso,1.6136,16.0517,26.53,35,0,0.99,CG,1703876273
358,358,Garoua Boulaï,5.8833,14.5500,21.92,27,0,1.88,CM,1703876286
397,397,Mount Isa,-20.7333,139.5000,24.87,61,0,0.00,AU,1703876302
452,452,Moussoro,13.6408,16.4901,23.42,18,0,4.33,TD,1703876049
534,534,Seeb,23.6703,58.1891,22.25,78,0,1.36,OM,1703876354


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = pd.DataFrame(filtered_city_data_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
103,Timbuktu,ML,16.7735,-3.0074,11,
119,Dapchi,NG,12.4904,11.4944,24,
165,Pisco,PE,-13.7000,-76.2167,69,
316,Duekoué,CI,6.7472,-7.3496,26,
324,Ouésso,CG,1.6136,16.0517,35,
358,Garoua Boulaï,CM,5.8833,14.5500,27,
397,Mount Isa,AU,-20.7333,139.5000,61,
452,Moussoro,TD,13.6408,16.4901,18,
534,Seeb,OM,23.6703,58.1891,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
#coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
search = "hotel"
radius = 10000
hotel_type = "hotel"

params = {
    "keyword": search,
    "radius": radius,
    "type": hotel_type,
    "key": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"{lat},{lng}"
    #params["bias"] = # YOUR CODE HERE
    
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Seeb - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
103,Timbuktu,ML,16.7735,-3.0074,11,
119,Dapchi,NG,12.4904,11.4944,24,
165,Pisco,PE,-13.7000,-76.2167,69,
316,Duekoué,CI,6.7472,-7.3496,26,
324,Ouésso,CG,1.6136,16.0517,35,
358,Garoua Boulaï,CM,5.8833,14.5500,27,
397,Mount Isa,AU,-20.7333,139.5000,61,
452,Moussoro,TD,13.6408,16.4901,18,
534,Seeb,OM,23.6703,58.1891,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

